In [45]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)

key = os.getenv('OPENAI_API_KEY')
tavily_key = os.getenv('TAVILY_API_KEY')

In [46]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages

In [47]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END

In [48]:
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_teddynote.tools.tavily import TavilySearch

In [49]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
import json

In [50]:
##### 1. 상태 정의 #####
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [51]:
##### 2. 도구 정의 및 바인딩 #####
tool = TavilySearch(max_results=1)      # 검색 도구
tools = [tool]                          # 도구 목록에 넣기

In [52]:
##### 3. LLM을 도구와 결합 #####
llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini'
)

llm_with_tools = llm.bind_tools(tools)  # 도구를 사용할 수 있는 llm

In [53]:
def chatbot(state: State):
    answer = llm_with_tools.invoke(state['messages'])
    return {'messages': [answer]}